In [2]:
import tensorflow as tf

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Softmax

### Create custom layers

In [7]:
# Create a custom layer

class MyLayer(Layer):
    
    def __init__(self, units, input_dim):
        super(MyLayer, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal')
        
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros')
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
dense_layer = MyLayer(3, 5)
x = tf.ones((1, 5))
print(dense_layer(x))
print(dense_layer.weights)

tf.Tensor([[-0.07445024  0.2630205   0.03668801]], shape=(1, 3), dtype=float32)
[<tf.Variable 'Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[-0.1183159 ,  0.07639796, -0.012381  ],
       [-0.01827505,  0.078046  ,  0.06065479],
       [ 0.01529051,  0.03106157, -0.10456753],
       [-0.09103274,  0.02787502,  0.01921523],
       [ 0.13788293,  0.04963992,  0.07376651]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


In [12]:
# Specify trainable or non-trainable weights

class MyLayer(Layer):
    
    def __init__(self, units, input_dim):
        super(MyLayer, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                trainable=False)
        
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros', 
                                trainable=False)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    
dense_layer = MyLayer(3, 5)

In [13]:
print('trainable weights:', len(dense_layer.trainable_weights))
print('non-trainable weights:', len(dense_layer.non_trainable_weights))

trainable weights: 0
non-trainable weights: 2


In [33]:
# Create a custom layer to accumulate means of output values

class MyLayerMean(Layer):
    
    def __init__(self, units, input_dim):
        super(MyLayerMean, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal')
        
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros')
        
        self.sum_activations = tf.Variable(initial_value=tf.zeros((units,)),
                                         trainable=False)
        self.number_call = tf.Variable(initial_value=0,
                                      trainable=False)
        
    def call(self, inputs):
        activations= tf.matmul(inputs, self.w) + self.b
        self.sum_activations.assign_add(tf.reduce_sum(activations,axis=0))
        self.number_call.assign_add(inputs.shape[0])
        return activations, self.sum_activations / tf.cast(self.number_call, tf.float32)
    
dense_layer = MyLayerMean(3, 5)

In [35]:
# Test the layer 

y, activation_means = dense_layer(tf.ones((1, 5)))
print(activation_means.numpy())

y, activation_means = dense_layer(tf.ones((1, 5)))
print(activation_means.numpy())

[ 0.04868636 -0.17261314 -0.143733  ]
[ 0.04868636 -0.17261314 -0.143733  ]


In [36]:
# Create a Dropout layer as a custom layer

class MyDropout(Layer):
    
    def __init__(self, rate):
        super(MyDropout, self).__init__()
        self.rate = rate
        
    def call(self, inputs):
        # Define the forward pass for dropout layer
        return tf.nn.dropout(inputs, rate=self.rate)

### Implement the custom layers into a model

In [41]:
# Build the model using custom layers with the model subclassing API

class MyModel(Model):
    
    def __init__(self, units_1, input_dim_1, units_2, units_3):
        super(MyModel, self).__init__()
        # Define layers
        self.layer_1 = MyLayer(units_1, input_dim_1)
        self.dropout_1 = MyDropout(0.5)
        self.layer_2 = MyLayer(units_2, units_1)
        self.dropout_2 = MyDropout(0.5)
        self.layer_3 = MyLayer(units_3, units_2)
        self.softmax = Softmax()
        
    def call(self, inputs):
        # Define a forward pass
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout_1(x)
        x = self.layer_2(x)
        x = tf.nn.relu(x)
        x = self.dropout_2(x)
        x = self.layer_3(x)
        return self.softmax(x)

In [42]:
# Instantiate a model object

model = MyModel(64, 10000, 64, 46)

print(model(tf.ones((1, 10000))))
model.summary()

tf.Tensor(
[[0.01342547 0.01076216 0.006756   0.00635154 0.01780686 0.01937537
  0.0159285  0.00991699 0.00385185 0.00578463 0.02210175 0.01263329
  0.01006382 0.04926752 0.00309126 0.00381934 0.01930514 0.02092838
  0.00144138 0.00489898 0.0300659  0.01368087 0.00276924 0.03548001
  0.01095282 0.02405936 0.00610227 0.01575533 0.01779753 0.03915828
  0.13950294 0.06217644 0.00560562 0.00274475 0.00479858 0.00811531
  0.00759054 0.01789135 0.00344876 0.16642314 0.05507303 0.01457846
  0.01503517 0.00963096 0.02262423 0.01142882]], shape=(1, 46), dtype=float32)
Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_layer_16 (MyLayer)        multiple                  640064    
_________________________________________________________________
my_dropout_4 (MyDropout)     multiple                  0         
_________________________________________________________________
my_layer_17 (MyLay